In [1]:
import stanza

# proxies = {'http': 'http://ip:port', 'https': 'http://ip:port'}
stanza.download('en')
nlp=  stanza.Pipeline('en')

2023-01-09 21:30:24 INFO: Downloading default packages for language: en (English) ...
2023-01-09 21:30:25 INFO: File exists: /Users/yixiantan/stanza_resources/en/default.zip
2023-01-09 21:30:28 INFO: Finished downloading models and saved to /Users/yixiantan/stanza_resources.
2023-01-09 21:30:28 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-01-09 21:30:30 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2023-01-09 21:30:30 INFO: Use device: cpu
2023-01-09 21:30:30 INFO: Loading: tokenize
2023-01-09 21:30:30 INFO: Loading: pos
2023-01-09 21:30:30 INFO: Loading: lemma
2023-01-09 21:30:30 INFO: Loading: depparse
2023-01-09 21:30:30 INFO: Loading: sentiment
2023-01-09 21:30:30 INFO: Loading: constituency
2023-01-09 21:30:30 INFO: Loading: ner
2023-01-09 21:30:31 INFO: Done loading processors!


In [39]:
doc = nlp("The cheap laptop is very slow.")
# doc.sentences[0].print_dependencies()
doc.sentences[0]

[
  {
    "id": 1,
    "text": "The",
    "lemma": "the",
    "upos": "DET",
    "xpos": "DT",
    "feats": "Definite=Def|PronType=Art",
    "head": 3,
    "deprel": "det",
    "start_char": 0,
    "end_char": 3,
    "ner": "O",
    "multi_ner": [
      "O"
    ]
  },
  {
    "id": 2,
    "text": "cheap",
    "lemma": "cheap",
    "upos": "ADJ",
    "xpos": "JJ",
    "feats": "Degree=Pos",
    "head": 3,
    "deprel": "amod",
    "start_char": 4,
    "end_char": 9,
    "ner": "O",
    "multi_ner": [
      "O"
    ]
  },
  {
    "id": 3,
    "text": "laptop",
    "lemma": "laptop",
    "upos": "NOUN",
    "xpos": "NN",
    "feats": "Number=Sing",
    "head": 6,
    "deprel": "nsubj",
    "start_char": 10,
    "end_char": 16,
    "ner": "O",
    "multi_ner": [
      "O"
    ]
  },
  {
    "id": 4,
    "text": "is",
    "lemma": "be",
    "upos": "AUX",
    "xpos": "VBZ",
    "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
    "head": 6,
    "deprel": "cop",
    "start_c

In [36]:
# Extracting adjectives
processed = doc.sentences[0]
adjectives = [word.lemma for word in doc.sentences[0].words if word.upos == "ADJ"]

In [37]:
print(adjectives)

['cheap']


## 50 random sentences from IMDB Dataset

In [73]:
from datasets import load_dataset

data_files = {"data": "imdb_dataset.csv"}
imdb_dataset = load_dataset("csv", data_files = data_files)

Using custom data configuration default-37f8cdc2103c6b6c
Found cached dataset csv (/Users/yixiantan/.cache/huggingface/datasets/csv/default-37f8cdc2103c6b6c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [74]:
import html

# Random sample
imdb_sample = imdb_dataset["data"].shuffle(seed=5)
imdb_sample["review"]

# Convert html unicode from reviews
imdb_dataset = imdb_dataset.map(lambda x: {"review": html.unescape(x["review"])})

# Remove line breaks
imdb_dataset = imdb_dataset.map(lambda x: {"review": x["review"].replace("<br />", " ")})

# # Replace apostrophes
# imdb_dataset = imdb_dataset.map(lambda x: {"review": x["review"].replace("\\'", "?")})

# Set all to lowercase
imdb_dataset = imdb_dataset.map(lambda x: {"review": x["review"].lower()})

Loading cached shuffled indices for dataset at /Users/yixiantan/.cache/huggingface/datasets/csv/default-37f8cdc2103c6b6c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-7cb455eb0777f4d7.arrow


  0%|          | 0/50000 [00:00<?, ?ex/s]

  0%|          | 0/50000 [00:00<?, ?ex/s]

  0%|          | 0/50000 [00:00<?, ?ex/s]

In [85]:
sentences = []

# Get all sentences in the dataset
for review in imdb_dataset["data"]["review"]:
    sentences.extend(review.split(". "))

sentences[:5]

["one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked",
 'they are right, as this is exactly what happened with me',
 ' the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go',
 'trust me, this is not a show for the faint hearted or timid',
 'this show pulls no punches with regards to drugs, sex or violence']